In [108]:
punctuations = '''“”!()-[]{};:'"\,<>./?@#$%^&*_~�।’‘1234567890''' #we can add suitable extra punctuation all the time
def remove_punctuation(d):
    review = d.replace('\n', '')
    no_punct = ""
    for char in review:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [77]:
def input_file(fileName):
    word = [[]]
    sentences = [[]]
    target = [[]]
    count_class = {}
    i=0
    with open(fileName, 'r') as file:
        for line in file:
            l = line.split()
            if len(l):
                if l[0] not in punctuations:
                    l[0] = remove_punctuation(l[0])
                    word[i].append((l[0], l[-1]))
                    target[i].append(l[-1])
                    sentences[i].append(l[0])
                    if l[-1] not in count_class.keys():
                        count_class.update({l[-1]:1})
                    else:
                        count_class[l[-1]] += 1
                else:
                    word[i].append((l[0], l[-1]))
                    sentences[i].append(l[0])
            else:
                word.append([])
                sentences.append([])
                target.append([])
                i+=1
    
    return word, sentences, target, count_class

In [78]:
train_token, train_sentence, train_target, count_class_train = input_file('train.txt')

In [79]:
dev_token, dev_sentence, dev_target, count_class_dev = input_file('dev.txt')

In [13]:
mxlen = max(list(len(x) for x in train_sentence))
mxlen

35

In [15]:
max(len(x) for x in dev_sentence)

33

In [17]:
vocab = train_sentence + dev_sentence
flat_list = [item for sublist in vocab for item in sublist]
unique_words = list(set(x for x in flat_list))
len_unique_words = len(unique_words)
len_unique_words

29175

In [18]:
labels = list(count_class_train.keys())
num_classes = len(labels)
num_classes

13

In [19]:
from gensim.models import Word2Vec
model_word_to_vec = Word2Vec(vocab, vector_size=100, window=5, min_count=1)

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

x = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in train_sentence]
x = pad_sequences(maxlen = mxlen+5, sequences=x, padding='post', value=len_unique_words-1)

In [27]:
tag2idx = {t: i for i,t in enumerate(labels)}
idx2tag = {i: t for i,t in enumerate(labels)}

In [37]:
y = [[tag2idx[w] for w in s] for s in train_target]
y = pad_sequences(maxlen = mxlen+5, sequences=y, padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes=num_classes) for i in y]

In [34]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import TimeDistributed, Bidirectional
input_word = Input(shape=(mxlen+5, 100))
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(input_word)
out = TimeDistributed(Dense(num_classes, activation='softmax'))(model)

model = Model(input_word, out)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 100)]         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 40, 200)          160800    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 40, 13)           2613      
 tributed)                                                       
                                                                 
Total params: 163,413
Trainable params: 163,413
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.config.run_functions_eagerly(True)

history = model.fit(
    x, np.array(y),
    validation_split=0.2,
    batch_size=32,
    epochs=3,
    verbose=1,
)

Epoch 1/3


/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


383/383 [==============================] - 385s 1s/step - loss: 0.3021 - accuracy: 0.9359 - val_loss: 0.1993 - val_accuracy: 0.9510
Epoch 2/3
383/383 [==============================] - 384s 1s/step - loss: 0.1934 - accuracy: 0.9519 - val_loss: 0.1815 - val_accuracy: 0.9536
Epoch 3/3
383/383 [==============================] - 385s 1s/step - loss: 0.1810 - accuracy: 0.9535 - val_loss: 0.1725 - val_accuracy: 0.9548


In [41]:
x_dev = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in dev_sentence]
x_dev = pad_sequences(maxlen = mxlen + 5, sequences=x_dev, padding='post', value=len_unique_words-1)

In [42]:
predict_dev = model.predict(x_dev)

25/25 [==============================] - 9s 375ms/step


In [44]:
l = [[]]
for i, sent in enumerate(dev_sentence):
    for j, word in enumerate(sent):
        l[i].append(np.argmax(predict_dev[i][j]))
    if i<len(dev_sentence)-1:
        l.append([])

In [45]:
dev_labels = [[]]
for i, sent in enumerate(dev_sentence):
    for j, word in enumerate(sent):
        dev_labels[i].append(idx2tag[l[i][j]])
    if i<len(dev_sentence)-1:
        dev_labels.append([])

In [46]:
with open('dev_pred_labels.txt', 'w') as f:
    for line in dev_labels:
        for word in line:
            f.write(f"{word}\n")
        f.write('\n')

In [47]:
import pickle

pickle.dump(model, open('Word2Vec-BiLSTM-model.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......bidirectional
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........layer
............cell
...............vars
............vars
.........vars
......input_layer
.........vars
......time_distributed
.........layer
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                

In [48]:
model_word_to_vec.save('word2vec.model')

In [50]:
def input_file_test(fileName):
    word = [[]]
    sentences = [[]]
    target = [[]]
    count_class = {}
    i=0
    with open(fileName, 'r') as file:
        for line in file:
            l = line.split()
            if len(l):
                if l[0] not in punctuations:
                    l[0] = remove_punctuation(l[0])
                    word[i].append(l[0])
                    sentences[i].append(l[0])
                else:
                    word[i].append((l[0], l[-1]))
                    sentences[i].append(l[0])
            else:
                word.append([])
                sentences.append([])
                i+=1
    
    return word, sentences

In [51]:
test_word, test_sentences = input_file_test('test.txt')

In [55]:
mxlen_test = max(list(len(x) for x in test_sentences))
mxlen_test

45

In [53]:
vocab = train_sentence + dev_sentence + test_sentences
flat_list = [item for sublist in vocab for item in sublist]
unique_words = list(set(x for x in flat_list))
len_unique_words = len(unique_words)
len_unique_words

42384

In [54]:
from gensim.models import Word2Vec
model_word_to_vec = Word2Vec(vocab, vector_size=100, window=5, min_count=1)

In [56]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

x = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in train_sentence]
x = pad_sequences(maxlen = 50, sequences=x, padding='post', value=len_unique_words-1)
y = [[tag2idx[w] for w in s] for s in train_target]
y = pad_sequences(maxlen = 50, sequences=y, padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes=num_classes) for i in y]

In [58]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import TimeDistributed, Bidirectional
input_word = Input(shape=(50, 100))
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(input_word)
out = TimeDistributed(Dense(num_classes, activation='softmax'))(model)

model = Model(input_word, out)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 50, 100)]         0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 50, 200)          160800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 50, 13)           2613      
 tributed)                                                       
                                                                 
Total params: 163,413
Trainable params: 163,413
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.config.run_functions_eagerly(True)

history = model.fit(
    x, np.array(y),
    validation_split=0.2,
    batch_size=32,
    epochs=3,
    verbose=1,
)

Epoch 1/3


/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


383/383 [==============================] - 468s 1s/step - loss: 0.2323 - accuracy: 0.9503 - val_loss: 0.1481 - val_accuracy: 0.9624
Epoch 2/3
345/383 [==========================>...] - ETA: 43s - loss: 0.1435 - accuracy: 0.9632

KeyboardInterrupt: 

In [62]:
vocab_td = train_sentence + dev_sentence
flat_list = [item for sublist in vocab_td for item in sublist]
unique_words = list(set(x for x in flat_list))
len_unique_words = len(unique_words)
len_unique_words

29175

array([[    0,     1,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       ...,
       [42383, 42383, 42383, ..., 42383, 42383, 42383],
       [42383, 42383, 42383, ..., 42383, 42383, 42383],
       [42383, 42383, 42383, ..., 42383, 42383, 42383]], dtype=int32)

In [65]:
train_sentence[0]

['তার',
 'মৃত্যুর',
 'দশ',
 'দিন',
 'পর',
 '১১৫',
 'কৃষ্ণাঙ্গ',
 'উচ্চ',
 'বিদ্যালয়ের',
 'শিক্ষার্থীরা',
 'তার',
 'হত্যার',
 'প্রতিবাদে',
 'ম্যাককম্ব',
 'এর',
 'মাধ্যমে',
 'মিছিল',
 'করেছে']

In [80]:
train_sentence.append(['unknown'])

In [81]:
dev_sentence.append(['unknown'])

In [82]:
train_target.append(['O'])

In [83]:
dev_target.append(['O'])

In [105]:
vocab_td = train_sentence + dev_sentence
flat_list = [item for sublist in vocab_td for item in sublist]
unique_words = list(set(x for x in flat_list))
len_unique_words = len(unique_words)
len_unique_words

29176

In [87]:
from gensim.models import Word2Vec
model_word_to_vec = Word2Vec(vocab_td, vector_size=100, window=5, min_count=1)

In [88]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

x = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in train_sentence]
x = pad_sequences(maxlen = 50, sequences=x, padding='post', value=len_unique_words-1)
y = [[tag2idx[w] for w in s] for s in train_target]
y = pad_sequences(maxlen = 50, sequences=y, padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes=num_classes) for i in y]

In [89]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import TimeDistributed, Bidirectional
input_word = Input(shape=(50, 100))
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(input_word)
out = TimeDistributed(Dense(num_classes, activation='softmax'))(model)

model = Model(input_word, out)
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

tf.config.run_functions_eagerly(True)

history = model.fit(
    x, np.array(y),
    validation_split=0.2,
    batch_size=32,
    epochs=3,
    verbose=1,
)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 50, 100)]         0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 50, 200)          160800    
 nal)                                                            
                                                                 
 time_distributed_4 (TimeDis  (None, 50, 13)           2613      
 tributed)                                                       
                                                                 
Total params: 163,413
Trainable params: 163,413
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3


/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


383/383 [==============================] - 480s 1s/step - loss: 0.3707 - accuracy: 0.9240 - val_loss: 0.1642 - val_accuracy: 0.9607
Epoch 2/3
383/383 [==============================] - 481s 1s/step - loss: 0.1584 - accuracy: 0.9609 - val_loss: 0.1481 - val_accuracy: 0.9624
Epoch 3/3
383/383 [==============================] - 479s 1s/step - loss: 0.1483 - accuracy: 0.9623 - val_loss: 0.1417 - val_accuracy: 0.9631


In [91]:
import pickle

pickle.dump(model, open('Word2Vec-BiLSTM-model-2.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......bidirectional
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........layer
............cell
...............vars
............vars
.........vars
......input_layer
.........vars
......time_distributed
.........layer
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                

In [90]:
model_word_to_vec.save('word2vec.model')

In [120]:
def input_file_test(fileName):
    word = [[]]
    sentences = [[]]
    target = [[]]
    count_class = {}
    i=0
    with open(fileName, 'r') as file:
        for line in file:
            l = line.split()
            if len(l):
                if l[0] not in punctuations:
                    l[0] = remove_punctuation(l[0])
                    if l[0] not in unique_words:
                        word[i].append('unknown')
                    else:
                        word[i].append(l[0])
                else:
                    if l[0] not in unique_words:
                        word[i].append('unknown')
                    else:
                        word[i].append(l[0])
            else:
                word.append([])
                i+=1
    
    return word

In [121]:
test_word = input_file_test('test.txt')

In [94]:
dev_token, dev_sentence, dev_target, count_class_dev = input_file('dev.txt')

In [97]:
x_dev = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in dev_sentence]
x_dev = pad_sequences(maxlen = 50, sequences=x_dev, padding='post', value=len_unique_words-1)

In [98]:
predict_dev = model.predict(x_dev)

25/25 [==============================] - 12s 463ms/step


In [100]:
l = [[]]
for i, sent in enumerate(dev_sentence):
    for j, word in enumerate(sent):
        l[i].append(np.argmax(predict_dev[i][j]))
    if i<len(dev_sentence)-1:
        l.append([])
        
dev_labels = [[]]
for i, sent in enumerate(dev_sentence):
    for j, word in enumerate(sent):
        dev_labels[i].append(idx2tag[l[i][j]])
    if i<len(dev_sentence)-1:
        dev_labels.append([])

with open('dev_pred.txt', 'w') as f:
    for line in dev_labels:
        for word in line:
            f.write(f"{word}\n")
        f.write('\n')

In [122]:
x_test = [[model_word_to_vec.wv.get_vector(x) for x in s] for s in test_word]
x_test = pad_sequences(maxlen = 50, sequences=x_test, padding='post', value=len_unique_words-1)

In [123]:
predict_dev = model.predict(x_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


413/413 [==============================] - 190s 460ms/step


In [126]:
l = [[]]
for i, sent in enumerate(test_word):
    for j, word in enumerate(sent):
        l[i].append(np.argmax(predict_dev[i][j]))
    if i<len(test_word)-1:
        l.append([])

dev_labels = [[]]
for i, sent in enumerate(test_word):
    for j, word in enumerate(sent):
        dev_labels[i].append(idx2tag[l[i][j]])
    if i<len(test_word)-1:
        dev_labels.append([])
        
with open('test_pred_labels.txt', 'w') as f:
    for line in dev_labels:
        for word in line:
            f.write(f"{word}\n")
        f.write('\n\n')